In [1]:
# Range dependent Hamiltonian Algorithm for numerical QUBO formulation
# 2 qubits examples

# 1st step original
import numpy as np
import random, math
import copy, sys

Dimension = 2
qubits = 2;
Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")  

Matrix A:
[[-26  -7]
 [-28 -16]]

Solution vector x:
[0 3]

Vector b:
[-21 -48]
# Matrix Q is
[[-2320.  5840.  1260.  2520.]
 [    0. -1720.  2520.  5040.]
 [    0.     0. -1525.  1220.]
 [    0.     0.     0. -2440.]]

Minimum energy is  -2745


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-2320.0, ('q2','q2'):-1720.0, ('q3','q3'):-1525.0, ('q4','q4'):-2440.0}

quadratic = {('q1','q2'):5840.0, ('q1','q3'):1260.0, ('q1','q4'):2520.0, ('q2','q3'):2520.0, ('q2','q4'):5040.0, ('q3','q4'):1220.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-2320.0, ('q2','q2'):-1720.0, ('q3','q3'):-1525.0, ('q4','q4'):-2440.0}

quadratic = {('q1','q2'):5840.0, ('q1','q3'):1260.0, ('q1','q4'):2520.0, ('q2','q3'):2520.0, ('q2','q4'):5040.0, ('q3','q4'):1220.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4  energy num_oc. chain_.
0  0  0  1  1 -2745.0     605     0.0
1  1  0  1  0 -2585.0     236     0.0
2  0  0  0  1 -2440.0      82     0.0
3  1  0  0  0 -2320.0      36     0.0
4  1  0  0  1 -2240.0      25     0.0
5  0  1  0  0 -1720.0      11     0.0
6  0  0  1  0 -1525.0       4     0.0
7  1  0  1  1 -1285.0       1     0.0
['BINARY', 8 rows, 1000 samples, 4 variables]


In [3]:
# 1st step subrange

Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")  

Extended solution vector x:
[-888471 -714455]

Vector b:
[28101431 36308468]
Translation vector T:
[-888472. -714456.]

Qubit notation:
[1. 1.]

Updated vector c:
[-33. -44.]
# Matrix Q is
[[-2720.  5840.  1260.  2520.]
 [    0. -2520.  2520.  5040.]
 [    0.     0. -1565.  1220.]
 [    0.     0.     0. -2520.]]

Minimum energy is  -3025.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-2720.0, ('q2','q2'):-2520.0, ('q3','q3'):-1565.0, ('q4','q4'):-2520.0}

quadratic = {('q1','q2'):5840.0, ('q1','q3'):1260.0, ('q1','q4'):2520.0, ('q2','q3'):2520.0, ('q2','q4'):5040.0, ('q3','q4'):1220.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [4]:
linear = {('q1','q1'):-2720.0, ('q2','q2'):-2520.0, ('q3','q3'):-1565.0, ('q4','q4'):-2520.0}

quadratic = {('q1','q2'):5840.0, ('q1','q3'):1260.0, ('q1','q4'):2520.0, ('q2','q3'):2520.0, ('q2','q4'):5040.0, ('q3','q4'):1220.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4  energy num_oc. chain_.
0  1  0  1  0 -3025.0     489     0.0
1  0  0  1  1 -2865.0     259     0.0
2  1  0  0  1 -2720.0     105     0.0
3  1  0  0  0 -2720.0      69     0.0
4  0  1  0  0 -2520.0      40     0.0
5  0  0  0  1 -2520.0      32     0.0
6  1  0  1  1 -1805.0       3     0.0
7  0  1  1  0 -1565.0       2     0.0
8  1  1  0  0   600.0       1     0.0
['BINARY', 9 rows, 1000 samples, 4 variables]


In [5]:
# 2nd step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[ -5  81]
 [-60 -42]]

Solution vector x:
[3 2]

Vector b:
[ 147 -264]
# Matrix Q is
[[-26585.  14500.   4230.   8460.]
 [     0. -45920.   8460.  16920.]
 [     0.      0. -37665.  33300.]
 [     0.      0.      0. -58680.]]

Minimum energy is  -91305


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-26585.0, ('q2','q2'):-45920.0, ('q3','q3'):-37665.0, ('q4','q4'):-58680.0}

quadratic = {('q1','q2'):14500.0, ('q1','q3'):4230.0, ('q1','q4'):8460.0, ('q2','q3'):8460.0, ('q2','q4'):16920.0, ('q3','q4'):33300.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [6]:
linear = {('q1','q1'):-26585.0, ('q2','q2'):-45920.0, ('q3','q3'):-37665.0, ('q4','q4'):-58680.0}

quadratic = {('q1','q2'):14500.0, ('q1','q3'):4230.0, ('q1','q4'):8460.0, ('q2','q3'):8460.0, ('q2','q4'):16920.0, ('q3','q4'):33300.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  1  1  0  1 -91305.0     945     0.0
6  1  1  0  1 -91305.0       1    0.25
1  0  1  0  1 -87680.0      32     0.0
2  0  1  1  1 -83585.0       3     0.0
3  1  1  1  1 -82980.0       2     0.0
4  1  1  1  0 -82980.0      15     0.0
5  1  0  0  1 -76805.0       2     0.0
['BINARY', 7 rows, 1000 samples, 4 variables]


In [7]:
# 2nd step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[-592292 -160788]

Vector b:
[-10062368  42290616]
Translation vector T:
[-592292. -160788.]

Qubit notation:
[0. 0.]

Updated vector c:
[0. 0.]
# Matrix Q is
[[ 3625. 14500.  4230.  8460.]
 [    0. 14500.  8460. 16920.]
 [    0.     0.  8325. 33300.]
 [    0.     0.     0. 33300.]]

Minimum energy is  -0.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):3625.0, ('q2','q2'):14500.0, ('q3','q3'):8325.0, ('q4','q4'):33300.0}

quadratic = {('q1','q2'):14500.0, ('q1','q3'):4230.0, ('q1','q4'):8460.0, ('q2','q3'):8460.0, ('q2','q4'):16920.0, ('q3','q4'):33300.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [8]:
linear = {('q1','q1'):3625.0, ('q2','q2'):14500.0, ('q3','q3'):8325.0, ('q4','q4'):33300.0}

quadratic = {('q1','q2'):14500.0, ('q1','q3'):4230.0, ('q1','q4'):8460.0, ('q2','q3'):8460.0, ('q2','q4'):16920.0, ('q3','q4'):33300.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 energy num_oc. chain_.
0  0  0  0  0    0.0     843     0.0
1  1  0  0  0 3625.0     145     0.0
2  0  0  1  0 8325.0      11     0.0
3  0  0  1  0 8325.0       1    0.25
['BINARY', 4 rows, 1000 samples, 4 variables]


In [9]:
# 3rd step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[  3  79]
 [-97 -29]]

Solution vector x:
[0 2]

Vector b:
[158 -58]
# Matrix Q is
[[ -2782.  37672.   6100.  12200.]
 [     0.  13272.  12200.  24400.]
 [     0.      0. -21246.  28328.]
 [     0.      0.      0. -28328.]]

Minimum energy is  -28328


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-2782.0, ('q2','q2'):13272.0, ('q3','q3'):-21246.0, ('q4','q4'):-28328.0}

quadratic = {('q1','q2'):37672.0, ('q1','q3'):6100.0, ('q1','q4'):12200.0, ('q2','q3'):12200.0, ('q2','q4'):24400.0, ('q3','q4'):28328.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [10]:
linear = {('q1','q1'):-2782.0, ('q2','q2'):13272.0, ('q3','q3'):-21246.0, ('q4','q4'):-28328.0}

quadratic = {('q1','q2'):37672.0, ('q1','q3'):6100.0, ('q1','q4'):12200.0, ('q2','q3'):12200.0, ('q2','q4'):24400.0, ('q3','q4'):28328.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  1 -28328.0     978     0.0
1  0  0  1  1 -21246.0       7     0.0
2  0  0  1  0 -21246.0       9     0.0
3  1  0  0  1 -18910.0       4     0.0
4  1  0  1  0 -17928.0       2     0.0
['BINARY', 5 rows, 1000 samples, 4 variables]


In [11]:
# 3rd step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[-398774  877411]

Vector b:
[68119147 13236159]
Translation vector T:
[-398776.  877408.]

Qubit notation:
[2. 3.]

Updated vector c:
[ 243. -281.]
# Matrix Q is
[[-46554.  37672.   6100.  12200.]
 [     0. -74272.  12200.  24400.]
 [     0.      0. -47610.  28328.]
 [     0.      0.      0. -81056.]]

Minimum energy is  -138010.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-46554.0, ('q2','q2'):-74272.0, ('q3','q3'):-47610.0, ('q4','q4'):-81056.0}

quadratic = {('q1','q2'):37672.0, ('q1','q3'):6100.0, ('q1','q4'):12200.0, ('q2','q3'):12200.0, ('q2','q4'):24400.0, ('q3','q4'):28328.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [12]:
linear = {('q1','q1'):-46554.0, ('q2','q2'):-74272.0, ('q3','q3'):-47610.0, ('q4','q4'):-81056.0}

quadratic = {('q1','q2'):37672.0, ('q1','q3'):6100.0, ('q1','q4'):12200.0, ('q2','q3'):12200.0, ('q2','q4'):24400.0, ('q3','q4'):28328.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  0  1  1  1 -138010.0     979     0.0
7  0  1  1  1 -138010.0       1    0.25
1  0  1  0  1 -130928.0      13     0.0
2  1  1  1  1 -128592.0       2     0.0
3  1  0  1  1 -128592.0       2     0.0
4  1  1  0  1 -127610.0       1     0.0
5  1  1  1  0 -112464.0       1     0.0
6  0  1  1  0 -109682.0       1     0.0
['BINARY', 8 rows, 1000 samples, 4 variables]


In [13]:
# 4th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[76 26]
 [43 93]]

Solution vector x:
[0 1]

Vector b:
[26 93]
# Matrix Q is
[[-4325. 30500. 11950. 23900.]
 [    0.  6600. 23900. 47800.]
 [    0.     0. -9325. 37300.]
 [    0.     0.     0.     0.]]

Minimum energy is  -9325


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-4325.0, ('q2','q2'):6600.0, ('q3','q3'):-9325.0, ('q4','q4'):0.0}

quadratic = {('q1','q2'):30500.0, ('q1','q3'):11950.0, ('q1','q4'):23900.0, ('q2','q3'):23900.0, ('q2','q4'):47800.0, ('q3','q4'):37300.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [14]:
linear = {('q1','q1'):-4325.0, ('q2','q2'):6600.0, ('q3','q3'):-9325.0, ('q4','q4'):0.0}

quadratic = {('q1','q2'):30500.0, ('q1','q3'):11950.0, ('q1','q4'):23900.0, ('q2','q3'):23900.0, ('q2','q4'):47800.0, ('q3','q4'):37300.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4  energy num_oc. chain_.
0  0  0  1  0 -9325.0     771     0.0
1  1  0  0  0 -4325.0     177     0.0
2  1  0  1  0 -1700.0      22     0.0
3  0  0  0  1     0.0      14     0.0
4  0  0  0  0     0.0      16     0.0
['BINARY', 5 rows, 1000 samples, 4 variables]


In [15]:
# 4th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[806222 155007]

Vector b:
[65303054 49083197]
Translation vector T:
[806220. 155004.]

Qubit notation:
[2. 3.]

Updated vector c:
[230. 365.]
# Matrix Q is
[[ -58725.   30500.   11950.   23900.]
 [      0. -102200.   23900.   47800.]
 [      0.       0.  -70525.   37300.]
 [      0.       0.       0. -122400.]]

Minimum energy is  -186125.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-58725.0, ('q2','q2'):-102200.0, ('q3','q3'):-70525.0, ('q4','q4'):-122400.0}

quadratic = {('q1','q2'):30500.0, ('q1','q3'):11950.0, ('q1','q4'):23900.0, ('q2','q3'):23900.0, ('q2','q4'):47800.0, ('q3','q4'):37300.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [16]:
linear = {('q1','q1'):-58725.0, ('q2','q2'):-102200.0, ('q3','q3'):-70525.0, ('q4','q4'):-122400.0}

quadratic = {('q1','q2'):30500.0, ('q1','q3'):11950.0, ('q1','q4'):23900.0, ('q2','q3'):23900.0, ('q2','q4'):47800.0, ('q3','q4'):37300.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  0  1  1  1 -186125.0     852     0.0
1  1  1  0  1 -181125.0     115     0.0
2  1  1  1  1 -178500.0      13     0.0
3  1  0  1  1 -178500.0       7     0.0
4  0  1  0  1 -176800.0      12     0.0
5  0  0  1  1 -155625.0       1     0.0
['BINARY', 6 rows, 1000 samples, 4 variables]


In [17]:
# 5th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[  3 -99]
 [-14  92]]

Solution vector x:
[1 3]

Vector b:
[-294  262]
# Matrix Q is
[[   9305.     820.   -3170.   -6340.]
 [      0.   19020.   -6340.  -12680.]
 [      0.       0.  -88155.   73060.]
 [      0.       0.       0. -139780.]]

Minimum energy is  -155080


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):9305.0, ('q2','q2'):19020.0, ('q3','q3'):-88155.0, ('q4','q4'):-139780.0}

quadratic = {('q1','q2'):820.0, ('q1','q3'):-3170.0, ('q1','q4'):-6340.0, ('q2','q3'):-6340.0, ('q2','q4'):-12680.0, ('q3','q4'):73060.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [18]:
linear = {('q1','q1'):9305.0, ('q2','q2'):19020.0, ('q3','q3'):-88155.0, ('q4','q4'):-139780.0}

quadratic = {('q1','q2'):820.0, ('q1','q3'):-3170.0, ('q1','q4'):-6340.0, ('q2','q3'):-6340.0, ('q2','q4'):-12680.0, ('q3','q4'):73060.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4    energy num_oc. chain_.
0  1  0  1  1 -155080.0     322     0.0
1  0  0  1  1 -154875.0     220     0.0
2  0  1  1  1 -154875.0     218     0.0
3  1  1  1  1 -154260.0     239     0.0
4  0  0  0  1 -139780.0       1     0.0
['BINARY', 5 rows, 1000 samples, 4 variables]


In [19]:
# 5th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[ 841744 -869676]

Vector b:
[ 88623156 -91794608]
Translation vector T:
[ 841744. -869676.]

Qubit notation:
[0. 0.]

Updated vector c:
[0. 0.]
# Matrix Q is
[[   205.    820.  -3170.  -6340.]
 [     0.    820.  -6340. -12680.]
 [     0.      0.  18265.  73060.]
 [     0.      0.      0.  73060.]]

Minimum energy is  -0.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):205.0, ('q2','q2'):820.0, ('q3','q3'):18265.0, ('q4','q4'):73060.0}

quadratic = {('q1','q2'):820.0, ('q1','q3'):-3170.0, ('q1','q4'):-6340.0, ('q2','q3'):-6340.0, ('q2','q4'):-12680.0, ('q3','q4'):73060.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [20]:
linear = {('q1','q1'):205.0, ('q2','q2'):820.0, ('q3','q3'):18265.0, ('q4','q4'):73060.0}

quadratic = {('q1','q2'):820.0, ('q1','q3'):-3170.0, ('q1','q4'):-6340.0, ('q2','q3'):-6340.0, ('q2','q4'):-12680.0, ('q3','q4'):73060.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4  energy num_oc. chain_.
0  0  0  0  0     0.0     439     0.0
1  1  0  0  0   205.0     295     0.0
2  0  1  0  0   820.0     183     0.0
3  1  1  0  0  1845.0      63     0.0
4  1  1  1  0 10600.0      10     0.0
5  0  1  1  0 12745.0       5     0.0
6  1  0  1  0 15300.0       2     0.0
7  0  0  1  0 18265.0       2     0.0
8  0  0  1  0 18265.0       1    0.25
['BINARY', 9 rows, 1000 samples, 4 variables]


In [21]:
# 6th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[-28  46]
 [-53  20]]

Solution vector x:
[2 1]

Vector b:
[-10 -86]
# Matrix Q is
[[ -6083.  14372.  -4696.  -9392.]
 [     0.  -4980.  -9392. -18784.]
 [     0.      0.   6876.  10064.]
 [     0.      0.      0.  18784.]]

Minimum energy is  -7496


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-6083.0, ('q2','q2'):-4980.0, ('q3','q3'):6876.0, ('q4','q4'):18784.0}

quadratic = {('q1','q2'):14372.0, ('q1','q3'):-4696.0, ('q1','q4'):-9392.0, ('q2','q3'):-9392.0, ('q2','q4'):-18784.0, ('q3','q4'):10064.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [22]:
linear = {('q1','q1'):-6083.0, ('q2','q2'):-4980.0, ('q3','q3'):6876.0, ('q4','q4'):18784.0}

quadratic = {('q1','q2'):14372.0, ('q1','q3'):-4696.0, ('q1','q4'):-9392.0, ('q2','q3'):-9392.0, ('q2','q4'):-18784.0, ('q3','q4'):10064.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4  energy num_oc. chain_.
0  0  1  1  0 -7496.0     550     0.0
1  1  0  0  0 -6083.0     270     0.0
2  1  1  0  1 -6083.0     123     0.0
3  0  1  0  1 -4980.0      26     0.0
4  0  1  0  0 -4980.0      14     0.0
5  1  0  1  0 -3903.0       7     0.0
6  1  1  1  0 -3903.0       6     0.0
7  1  1  1  1 -3231.0       2     0.0
8  0  0  0  0     0.0       2     0.0
['BINARY', 9 rows, 1000 samples, 4 variables]


In [23]:
# 6th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[393446 194290]

Vector b:
[ -2079148 -16966838]
Translation vector T:
[393444. 194288.]

Qubit notation:
[2. 2.]

Updated vector c:
[ 36. -66.]
# Matrix Q is
[[ -1387.  14372.  -4696.  -9392.]
 [     0.   4412.  -9392. -18784.]
 [     0.      0.   1844.  10064.]
 [     0.      0.      0.   8720.]]

Minimum energy is  -5652.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-1387.0, ('q2','q2'):4412.0, ('q3','q3'):1844.0, ('q4','q4'):8720.0}

quadratic = {('q1','q2'):14372.0, ('q1','q3'):-4696.0, ('q1','q4'):-9392.0, ('q2','q3'):-9392.0, ('q2','q4'):-18784.0, ('q3','q4'):10064.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [24]:
linear = {('q1','q1'):-1387.0, ('q2','q2'):4412.0, ('q3','q3'):1844.0, ('q4','q4'):8720.0}

quadratic = {('q1','q2'):14372.0, ('q1','q3'):-4696.0, ('q1','q4'):-9392.0, ('q2','q3'):-9392.0, ('q2','q4'):-18784.0, ('q3','q4'):10064.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4  energy num_oc. chain_.
0  0  1  0  1 -5652.0     711     0.0
1  1  0  1  0 -4239.0     113     0.0
2  1  1  1  1 -4239.0     120     0.0
3  0  1  1  0 -3136.0      21     0.0
4  0  1  1  1 -3136.0      17     0.0
5  1  1  0  1 -2059.0       5     0.0
6  1  0  0  1 -2059.0      11     0.0
7  1  0  0  0 -1387.0       2     0.0
['BINARY', 8 rows, 1000 samples, 4 variables]


In [25]:
# 7th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[-34 -95]
 [ 49  87]]

Solution vector x:
[0 2]

Vector b:
[-190  174]
# Matrix Q is
[[-26415.  14228.  14986.  29972.]
 [     0. -45716.  29972.  59944.]
 [     0.      0. -49782.  66376.]
 [     0.      0.      0. -66376.]]

Minimum energy is  -66376


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-26415.0, ('q2','q2'):-45716.0, ('q3','q3'):-49782.0, ('q4','q4'):-66376.0}

quadratic = {('q1','q2'):14228.0, ('q1','q3'):14986.0, ('q1','q4'):29972.0, ('q2','q3'):29972.0, ('q2','q4'):59944.0, ('q3','q4'):66376.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [26]:
linear = {('q1','q1'):-26415.0, ('q2','q2'):-45716.0, ('q3','q3'):-49782.0, ('q4','q4'):-66376.0}

quadratic = {('q1','q2'):14228.0, ('q1','q3'):14986.0, ('q1','q4'):29972.0, ('q2','q3'):29972.0, ('q2','q4'):59944.0, ('q3','q4'):66376.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  1 -66376.0     366     0.0
1  0  1  1  0 -65526.0     442     0.0
2  1  0  0  1 -62819.0      39     0.0
3  1  1  1  0 -62727.0      77     0.0
4  1  0  1  0 -61211.0      43     0.0
5  1  1  0  0 -57903.0      19     0.0
6  0  1  0  1 -52148.0      10     0.0
7  0  0  1  1 -49782.0       1     0.0
8  0  0  1  0 -49782.0       3     0.0
['BINARY', 9 rows, 1000 samples, 4 variables]


In [27]:
# 7th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[ 997057 -867238]

Vector b:
[ 48487672 -26593913]
Translation vector T:
[ 997056. -867240.]

Qubit notation:
[1. 2.]

Updated vector c:
[-224.  223.]
# Matrix Q is
[[-33529.  14228.  14986.  29972.]
 [     0. -59944.  29972.  59944.]
 [     0.      0. -64768.  66376.]
 [     0.      0.      0. -96348.]]

Minimum energy is  -99905.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-33529.0, ('q2','q2'):-59944.0, ('q3','q3'):-64768.0, ('q4','q4'):-96348.0}

quadratic = {('q1','q2'):14228.0, ('q1','q3'):14986.0, ('q1','q4'):29972.0, ('q2','q3'):29972.0, ('q2','q4'):59944.0, ('q3','q4'):66376.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [28]:
linear = {('q1','q1'):-33529.0, ('q2','q2'):-59944.0, ('q3','q3'):-64768.0, ('q4','q4'):-96348.0}

quadratic = {('q1','q2'):14228.0, ('q1','q3'):14986.0, ('q1','q4'):29972.0, ('q2','q3'):29972.0, ('q2','q4'):59944.0, ('q3','q4'):66376.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  1  0  0  1 -99905.0     228     0.0
1  1  1  1  0 -99055.0     399     0.0
2  0  0  0  1 -96348.0     123     0.0
3  0  1  0  1 -96348.0      99     0.0
4  0  1  1  0 -94740.0      85     0.0
5  0  0  1  1 -94740.0      58     0.0
6  1  1  0  1 -85677.0       2     0.0
7  1  0  1  1 -83311.0       2     0.0
8  1  0  1  0 -83311.0       2     0.0
9  1  1  0  0 -79245.0       2     0.0
['BINARY', 10 rows, 1000 samples, 4 variables]


In [29]:
# 8th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[-43 -91]
 [-71 -93]]

Solution vector x:
[1 0]

Vector b:
[-43 -71]
# Matrix Q is
[[-6890. 27560. 21032. 42064.]
 [    0.     0. 42064. 84128.]
 [    0.     0. -4102. 67720.]
 [    0.     0.     0. 25656.]]

Minimum energy is  -6890


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-6890.0, ('q2','q2'):0.0, ('q3','q3'):-4102.0, ('q4','q4'):25656.0}

quadratic = {('q1','q2'):27560.0, ('q1','q3'):21032.0, ('q1','q4'):42064.0, ('q2','q3'):42064.0, ('q2','q4'):84128.0, ('q3','q4'):67720.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [30]:
linear = {('q1','q1'):-6890.0, ('q2','q2'):0.0, ('q3','q3'):-4102.0, ('q4','q4'):25656.0}

quadratic = {('q1','q2'):27560.0, ('q1','q3'):21032.0, ('q1','q4'):42064.0, ('q2','q3'):42064.0, ('q2','q4'):84128.0, ('q3','q4'):67720.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4  energy num_oc. chain_.
0  1  0  0  0 -6890.0     441     0.0
1  0  0  1  0 -4102.0     362     0.0
2  0  0  0  0     0.0      64     0.0
3  0  1  0  0     0.0     126     0.0
4  1  0  1  0 10040.0       5     0.0
5  1  1  0  0 20670.0       2     0.0
['BINARY', 6 rows, 1000 samples, 4 variables]


In [31]:
# 8th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[322256  56730]

Vector b:
[-19019438 -28156066]
Translation vector T:
[322256.  56728.]

Qubit notation:
[0. 2.]

Updated vector c:
[-182. -186.]
# Matrix Q is
[[-35174.  27560.  21032.  42064.]
 [     0. -56568.  42064.  84128.]
 [     0.      0. -50790.  67720.]
 [     0.      0.      0. -67720.]]

Minimum energy is  -67720.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-35174.0, ('q2','q2'):-56568.0, ('q3','q3'):-50790.0, ('q4','q4'):-67720.0}

quadratic = {('q1','q2'):27560.0, ('q1','q3'):21032.0, ('q1','q4'):42064.0, ('q2','q3'):42064.0, ('q2','q4'):84128.0, ('q3','q4'):67720.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [32]:
linear = {('q1','q1'):-35174.0, ('q2','q2'):-56568.0, ('q3','q3'):-50790.0, ('q4','q4'):-67720.0}

quadratic = {('q1','q2'):27560.0, ('q1','q3'):21032.0, ('q1','q4'):42064.0, ('q2','q3'):42064.0, ('q2','q4'):84128.0, ('q3','q4'):67720.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  0  0  1 -67720.0     343     0.0
1  0  1  1  0 -65294.0     310     0.0
2  1  0  1  0 -64932.0     177     0.0
3  1  1  0  0 -64182.0     109     0.0
4  1  0  0  1 -60830.0      38     0.0
5  0  1  0  0 -56568.0       6     0.0
6  1  1  1  0 -51876.0       5     0.0
7  0  0  1  1 -50790.0       9     0.0
8  0  0  1  0 -50790.0       3     0.0
['BINARY', 9 rows, 1000 samples, 4 variables]


In [33]:
# 9th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[  7  66]
 [-94  76]]

Solution vector x:
[1 3]

Vector b:
[205 134]
# Matrix Q is
[[ 31207.  35540. -13364. -26728.]
 [     0.  80184. -26728. -53456.]
 [     0.      0. -37296.  40528.]
 [     0.      0.      0. -54328.]]

Minimum energy is  -59981


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):31207.0, ('q2','q2'):80184.0, ('q3','q3'):-37296.0, ('q4','q4'):-54328.0}

quadratic = {('q1','q2'):35540.0, ('q1','q3'):-13364.0, ('q1','q4'):-26728.0, ('q2','q3'):-26728.0, ('q2','q4'):-53456.0, ('q3','q4'):40528.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [34]:
linear = {('q1','q1'):31207.0, ('q2','q2'):80184.0, ('q3','q3'):-37296.0, ('q4','q4'):-54328.0}

quadratic = {('q1','q2'):35540.0, ('q1','q3'):-13364.0, ('q1','q4'):-26728.0, ('q2','q3'):-26728.0, ('q2','q4'):-53456.0, ('q3','q4'):40528.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  1  0  1  1 -59981.0     826     0.0
1  0  0  0  1 -54328.0     133     0.0
2  0  0  1  1 -51096.0      17     0.0
3  0  1  1  1 -51096.0       9     0.0
4  1  0  0  1 -49849.0      15     0.0
['BINARY', 5 rows, 1000 samples, 4 variables]


In [35]:
# 9th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[-886911 -753065]

Vector b:
[-55910667  26136694]
Translation vector T:
[-886912. -753068.]

Qubit notation:
[1. 3.]

Updated vector c:
[205. 134.]
# Matrix Q is
[[ 31207.  35540. -13364. -26728.]
 [     0.  80184. -26728. -53456.]
 [     0.      0. -37296.  40528.]
 [     0.      0.      0. -54328.]]

Minimum energy is  -59981.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):31207.0, ('q2','q2'):80184.0, ('q3','q3'):-37296.0, ('q4','q4'):-54328.0}

quadratic = {('q1','q2'):35540.0, ('q1','q3'):-13364.0, ('q1','q4'):-26728.0, ('q2','q3'):-26728.0, ('q2','q4'):-53456.0, ('q3','q4'):40528.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [36]:
linear = {('q1','q1'):31207.0, ('q2','q2'):80184.0, ('q3','q3'):-37296.0, ('q4','q4'):-54328.0}

quadratic = {('q1','q2'):35540.0, ('q1','q3'):-13364.0, ('q1','q4'):-26728.0, ('q2','q3'):-26728.0, ('q2','q4'):-53456.0, ('q3','q4'):40528.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  1  0  1  1 -59981.0     764     0.0
1  0  0  0  1 -54328.0     158     0.0
2  0  1  1  1 -51096.0      38     0.0
3  0  0  1  1 -51096.0      28     0.0
4  1  0  0  1 -49849.0      10     0.0
5  0  0  1  0 -37296.0       2     0.0
['BINARY', 6 rows, 1000 samples, 4 variables]


In [37]:
# 10th step original

Range_original = pow(2,qubits) 
Range_coefficient = 100;
A = np.random.randint(-Range_coefficient, Range_coefficient, size=(Dimension,Dimension))
np.set_printoptions(threshold=sys.maxsize)
print("Matrix A:")
print(A)

x_fin = np.random.randint(0, Range_original, size=Dimension)
print("\nSolution vector x:")
print(x_fin)

b = np.dot(A, x_fin)
print("\nVector b:")
print(b)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Matrix A:
[[-41 -40]
 [-57  -8]]

Solution vector x:
[3 1]

Vector b:
[-163 -179]
# Matrix Q is
[[-28842.  19720.   4192.   8384.]
 [     0. -47824.   8384.  16768.]
 [     0.      0. -14240.   6656.]
 [     0.      0.      0. -25152.]]

Minimum energy is  -58610


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-28842.0, ('q2','q2'):-47824.0, ('q3','q3'):-14240.0, ('q4','q4'):-25152.0}

quadratic = {('q1','q2'):19720.0, ('q1','q3'):4192.0, ('q1','q4'):8384.0, ('q2','q3'):8384.0, ('q2','q4'):16768.0, ('q3','q4'):6656.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [38]:
linear = {('q1','q1'):-28842.0, ('q2','q2'):-47824.0, ('q3','q3'):-14240.0, ('q4','q4'):-25152.0}

quadratic = {('q1','q2'):19720.0, ('q1','q3'):4192.0, ('q1','q4'):8384.0, ('q2','q3'):8384.0, ('q2','q4'):16768.0, ('q3','q4'):6656.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  1  1  1  0 -58610.0     707     0.0
1  1  1  0  1 -56946.0     107     0.0
2  1  1  0  0 -56946.0     120     0.0
3  0  1  0  1 -56208.0      50     0.0
4  0  1  1  1 -55408.0      14     0.0
5  0  1  1  0 -53680.0       2     0.0
['BINARY', 6 rows, 1000 samples, 4 variables]


In [39]:
# 10th step subrange
Range_ext = pow(2,20)
x_ext = np.random.randint(-Range_ext, Range_ext, size=Dimension)
print("Extended solution vector x:")
print(x_ext)

b = np.dot(A, x_ext)
print("\nVector b:")
print(b)

# Calculate translation number including solution vector x
Trs = np.zeros(Dimension)
x_qubit = np.zeros(Dimension)
sub_size = pow(2,qubits)
for k in range(Dimension):
    Val = divmod(x_ext[k], sub_size)
    Trs[k] = sub_size*Val[0]
    x_qubit[k] = Val[1]
print("Translation vector T:")
print(Trs)
print("\nQubit notation:")
print(x_qubit)
c = b - np.dot(A, Trs)
print("\nUpdated vector c:")
print(c)

QM = np.zeros((qubits*Dimension, qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*c[k]
            po = qubits*i + l
            QM[po][po] = QM[po][po] + cef1 - cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = qubits*i + l1
                po2 = qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = qubits*i + l1
                    po2 = qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(c,c))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
max_d = format(len(str(qubits*Dimension)), '02')
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(qubits*Dimension-1): 
    linear = format(i+1, max_d)
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",qubits*Dimension,"','q",qubits*Dimension,"'):",format(QM[qubits*Dimension-1][qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(qubits*Dimension-1):
    for j in range(i+1,qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = format(i+1, max_d)
            qdrt2 = format(j+1, max_d)
            if i == qubits*Dimension-2 and j == qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")


Extended solution vector x:
[-736030  489614]

Vector b:
[10592670 38036798]
Translation vector T:
[-736032.  489612.]

Qubit notation:
[2. 2.]

Updated vector c:
[-162. -130.]
# Matrix Q is
[[-23174.  19720.   4192.   8384.]
 [     0. -36488.   8384.  16768.]
 [     0.      0. -13376.   6656.]
 [     0.      0.      0. -23424.]]

Minimum energy is  -43144.0


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-23174.0, ('q2','q2'):-36488.0, ('q3','q3'):-13376.0, ('q4','q4'):-23424.0}

quadratic = {('q1','q2'):19720.0, ('q1','q3'):4192.0, ('q1','q4'):8384.0, ('q2','q3'):8384.0, ('q2','q4'):16768.0, ('q3','q4'):6656.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [40]:
linear = {('q1','q1'):-23174.0, ('q2','q2'):-36488.0, ('q3','q3'):-13376.0, ('q4','q4'):-23424.0}

quadratic = {('q1','q2'):19720.0, ('q1','q3'):4192.0, ('q1','q4'):8384.0, ('q2','q3'):8384.0, ('q2','q4'):16768.0, ('q3','q4'):6656.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4   energy num_oc. chain_.
0  0  1  0  1 -43144.0     687     0.0
1  0  1  1  0 -41480.0     181     0.0
2  0  1  1  1 -41480.0      39     0.0
3  1  0  1  1 -40742.0      21     0.0
4  1  1  1  0 -40742.0      47     0.0
5  1  1  0  0 -39942.0      24     0.0
6  1  0  0  1 -38214.0       1     0.0
['BINARY', 7 rows, 1000 samples, 4 variables]
